In [2]:
%%capture
!pip install contextualized-topic-models==2.2.0

In [3]:
%%capture
!pip install pyldavis

In [4]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 24.55 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 63.6 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-11-27 11:16:30--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (b

# 필요한 것 임포트

In [1]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm
import pandas as pd

# 데이터 불러오기

In [3]:
captionTable = pd.read_csv('/content/drive/MyDrive/kusw_second_project/captionTable.csv')

In [4]:
captionTable

,Unnamed: 0,video_id,start_ms,end_ms,timetext,captions
0,0,arj7oStGLkU,0,7000,0:00,번역: Jihyeon J. Kim 검토: Ju Hye Lim
1,1,arj7oStGLkU,12645,14015,0:12,대학에 다닐 때
2,2,arj7oStGLkU,15349,16913,0:15,저는 정치학을 전공했습니다.
3,3,arj7oStGLkU,16937,19399,0:16,그래서 작성해야 하는 과제가 많았죠.
4,4,arj7oStGLkU,19423,23911,0:19,보통의 학생은 과제를 할 때 이런 식으로 일을 분산할 겁니다.
...,...,...,...,...,...,...
253358,253358,mw2z9lV3W1g,1065793,1070377,17:45,우리는 그런 공화국을 잃어버렸습니다.
253359,253359,mw2z9lV3W1g,1070378,1074726,17:50,우리 모두 공화국을 되찾기 위해 행동해야 합니다.
253360,253360,mw2z9lV3W1g,1074726,1076370,17:54,정말 고맙습니다.
253361,253361,mw2z9lV3W1g,1076370,1082423,17:56,(박수)


# 전처리

In [8]:
video_id_list = list(set(captionTable['video_id']))

In [18]:
doc_0 = captionTable[captionTable['video_id'] == video_id_list[0]]

In [28]:
type(doc_0)

pandas.core.frame.DataFrame

In [27]:
' '.join(list(doc_0['captions'][1:]))

'오늘날 무엇이 좋은 지도자를 만듭니까? 우리 중 대다수는 앞장서서 지휘하고 그의 추종자들을 보호하는 영웅같은 모습을 떠올립니다. 하지만 그런 모습은 다른 세대의 이미지이고, 현재의 세상이나 앞으로 다가올 세상이 아닌, 과거의 성공에서 비롯된 지도력 개발 프로그램들 또한 시대에 뒤쳐졌습니다 저희는 4천개의 회사들에게 자사의 지도력 개발 프로그램의 효과가 어떻냐고 물었습니다. 그 중 58%는 중대한 지도자 역할에 대한 큰 재능 차이를 언급했습니다. 이 말은 즉 기업 연수 프로그램이나 현장학습, 평가, 지도, 이 모든 것에도 불구하고 반 이상의 회사들이 훌륭한 지도자를 충분히  길러내지 못 했습니다. 여러분은 자신에게 이렇게 물을 수 있어요. 우리 회사는 나를 21세기의 지도자로 키울 수 있는가? 아마 아닐 겁니다. 저는 25 년 동안 일을 하면서 무엇이 위대한 지도자를 만드는지 관찰해왔습니다. 전 포춘 500 기업 내에서 일했고 200명 이상의 최고 경영자들에게 조언을 해줬으며 여러분이 상상하시는 것보다 더 많은 지도자들과 인맥을 일구어 왔습니다. 하지만 몇년 전, 저는 지도자 후보들 중에서 충격적인 추세를 알아챘습니다. 모든 노력에도 불구하고 개개인에 대한 비슷한 이야기가 계속 떠올랐던 겁니다. 한 이야기는 크리스에 관한 것이었어요. 잠재력이 큰 슈퍼스타 지도자였는데 새로운 부서로 가서는 실패를 해서 회복이 불가능할 정도로 손실을 끼쳤습니다. 최고 경영자인 시드니처럼 좌절감을 느끼는 경우도 있습니다. 자신의 회사가 지도자를 위한 최고의 회사로 꼽혔지만 상위 50명의 지도자 중 단 한 명 만이 중요한 계획을 이끌어나갈  준비가 되었기 때문이죠. 또다른 이야기가 있습니다. 한때 잘 나가던 사업을 이끈 고위 지도자 팀이 시장의 변화를 따라잡지 못해서 어쩔 수 없이 회사의 규모를 반으로 줄이거나 폐업한 경우도 있습니다. 이렇게 되풀이되는 사례들로 인해 전 두 가지 질문을 가지게 되었습니다. 지도력 개발에 엄청난 투자가 있음에도 불구하고 왜 지도력의 차이가 벌어지는 것

In [29]:
def line2doc(video_id_idx):
  doc_df = captionTable[captionTable['video_id'] == video_id_list[video_id_idx]]
  return ' '.join(list(doc_df['captions'][1:]))

In [32]:
doc_list = list(map(line2doc, range(len(video_id_list))))

In [35]:
doc_list[0]

'오늘날 무엇이 좋은 지도자를 만듭니까? 우리 중 대다수는 앞장서서 지휘하고 그의 추종자들을 보호하는 영웅같은 모습을 떠올립니다. 하지만 그런 모습은 다른 세대의 이미지이고, 현재의 세상이나 앞으로 다가올 세상이 아닌, 과거의 성공에서 비롯된 지도력 개발 프로그램들 또한 시대에 뒤쳐졌습니다 저희는 4천개의 회사들에게 자사의 지도력 개발 프로그램의 효과가 어떻냐고 물었습니다. 그 중 58%는 중대한 지도자 역할에 대한 큰 재능 차이를 언급했습니다. 이 말은 즉 기업 연수 프로그램이나 현장학습, 평가, 지도, 이 모든 것에도 불구하고 반 이상의 회사들이 훌륭한 지도자를 충분히  길러내지 못 했습니다. 여러분은 자신에게 이렇게 물을 수 있어요. 우리 회사는 나를 21세기의 지도자로 키울 수 있는가? 아마 아닐 겁니다. 저는 25 년 동안 일을 하면서 무엇이 위대한 지도자를 만드는지 관찰해왔습니다. 전 포춘 500 기업 내에서 일했고 200명 이상의 최고 경영자들에게 조언을 해줬으며 여러분이 상상하시는 것보다 더 많은 지도자들과 인맥을 일구어 왔습니다. 하지만 몇년 전, 저는 지도자 후보들 중에서 충격적인 추세를 알아챘습니다. 모든 노력에도 불구하고 개개인에 대한 비슷한 이야기가 계속 떠올랐던 겁니다. 한 이야기는 크리스에 관한 것이었어요. 잠재력이 큰 슈퍼스타 지도자였는데 새로운 부서로 가서는 실패를 해서 회복이 불가능할 정도로 손실을 끼쳤습니다. 최고 경영자인 시드니처럼 좌절감을 느끼는 경우도 있습니다. 자신의 회사가 지도자를 위한 최고의 회사로 꼽혔지만 상위 50명의 지도자 중 단 한 명 만이 중요한 계획을 이끌어나갈  준비가 되었기 때문이죠. 또다른 이야기가 있습니다. 한때 잘 나가던 사업을 이끈 고위 지도자 팀이 시장의 변화를 따라잡지 못해서 어쩔 수 없이 회사의 규모를 반으로 줄이거나 폐업한 경우도 있습니다. 이렇게 되풀이되는 사례들로 인해 전 두 가지 질문을 가지게 되었습니다. 지도력 개발에 엄청난 투자가 있음에도 불구하고 왜 지도력의 차이가 벌어지는 것

In [36]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [103]:
class CustomTokenizer_nv:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.pos(sent)
        result = [word[0] for word in word_tokens if (word[1].startswith('N') or word[1].startswith('VA')) and len(word[0]) > 1]
        return result

In [104]:
custom_tokenizer = CustomTokenizer_nv(Mecab())

In [38]:
# ma = Mecab()

In [ ]:
# [word for word in ma.pos(doc_list[0]) if (word[1].startswith('N') or word[1].startswith('VA')) and len(word[0]) > 1]

In [ ]:
# [word for word in ma.morphs(doc_list[0]) if len(word) > 1]

In [105]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [106]:
train_bow_embeddings = vectorizer.fit_transform(doc_list)

In [94]:
print(train_bow_embeddings.shape)

(918, 3000)


In [107]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
vocab

In [97]:
len(vocab)

3000

In [98]:
train_contextualized_embeddings = bert_embeddings_from_list(doc_list, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [108]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [109]:
train_contextualized_embeddings.shape

(918, 768)

In [71]:
train_bow_embeddings

<918x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 372813 stored elements in Compressed Sparse Row format>

In [74]:
len(id2token)

3000

## Combined TM 학습하기
이제 토픽 모델을 학습합니다. 여기서는 하이퍼파라미터에 해당하는 토픽의 개수(n_components)로는 50개를 선정합니다.

In [113]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=10, num_epochs=30)
ctm.fit(training_dataset)

Epoch: [30/30]	 Seen Samples: [27540/27540]	Train Loss: 4558.797045206971	Time: 0:00:00.557443: : 30it [00:17,  1.75it/s]


# 토픽들

학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.

```
get_topic_lists
```
해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [114]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['성능', '관측', '물리학', '실현', '화성'],
             1: ['웃음', '이야기', '자신', '아이', '그녀'],
             2: ['사회', '때문', '사람', '가지', '여러분'],
             3: ['가능', '여기', '이것', '방법', '하나'],
             4: ['사람', '때문', '사회', '세계', '우리'],
             5: ['우리', '사람', '여러분', '때문', '세계'],
             6: ['가족', '존중', '친구', '기도', '로스'],
             7: ['파이', '재단', '저것', '텐데요', '착각'],
             8: ['안녕', '엔지니어', '급진', '저서', '종말'],
             9: ['부위', '효과', '건물', '한때', '감염']})

# 시각화

우리의 토픽들을 시각화하기 위해서는 PyLDAvis를 사용합니다.

위에서 출력한 토픽 번호는 pyLDAvis에서 할당한 토픽 번호와 일치하지 않으므로 주의합시다.  

In [ ]:
import pyLDAvis as vis

In [115]:
lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [00:05,  1.98it/s]
